## Libraries used for EDA

In [ ]:
import pandas as pd #data manipulation and analysis
import matplotlib.pyplot as plt #visualizations
import re

## Overview of the Dataset

In [ ]:
listings= pd.read_csv('/content/drive/MyDrive/Airbnb/Airbnb_original_data/listings2024.csv') #Load the dataset
listings.head() #Display few rows

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,31094,https://www.airbnb.com/rooms/31094,20240629155650,2024-06-30,previous scrape,"Beautiful, spacious, central, renovated Penthouse","Welcome to our home, we hope you will enjoy Wo...","What else is nearby?<br />To be honest, We thi...",https://a0.muscache.com/pictures/miso/Hosting-...,129976,...,4.82,4.80,4.53,NaN,f,1,1,0,0,0.11
1,262961,https://www.airbnb.com/rooms/262961,20240629155650,2024-06-30,city scrape,192m2 FLAT+ 8m2 BALCONY IN CENTER NØRREBRO HOOD,If you are looking for a large apartment in ce...,TIME OUT - The world’s coolest neighbourhoods ...,https://a0.muscache.com/pictures/261aa506-7b13...,1379904,...,4.94,4.89,4.53,NaN,f,1,1,0,0,0.24
2,263036,https://www.airbnb.com/rooms/263036,20240629155650,2024-06-30,city scrape,Bright flat in central location,NaN,NaN,https://a0.muscache.com/pictures/17770169/f1fb...,1232471,...,4.86,4.81,4.67,NaN,f,1,1,0,0,0.14
3,32379,https://www.airbnb.com/rooms/32379,20240629155650,2024-06-30,city scrape,"155 m2 artist flat on Vesterbro, with 2 bathrooms",You enter a narrow entrance and feel the good ...,"Værnedamsvej area is super hip area, we call i...",https://a0.muscache.com/pictures/miso/Hosting-...,140105,...,4.91,4.89,4.71,NaN,f,2,1,1,0,0.49
4,263708,https://www.airbnb.com/rooms/263708,20240629155650,2024-06-30,city scrape,Urban garden on Vesterbro rooftop,NaN,NaN,https://a0.muscache.com/pictures/8ce36f40-ac59...,1383888,...,5.00,4.67,4.00,NaN,f,1,1,0,0,0.02


In [ ]:
print(listings.shape) #Understand the dimension of the dataset

(20909, 75)


In [ ]:
print(listings.info()) #Understand the structure of the dataset: its dimensions, columns and data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20909 entries, 0 to 20908
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            20909 non-null  int64  
 1   listing_url                                   20909 non-null  object 
 2   scrape_id                                     20909 non-null  int64  
 3   last_scraped                                  20909 non-null  object 
 4   source                                        20909 non-null  object 
 5   name                                          20909 non-null  object 
 6   description                                   20235 non-null  object 
 7   neighborhood_overview                         8984 non-null   object 
 8   picture_url                                   20909 non-null  object 
 9   host_id                                       20909 non-null 

## Price Variable

In [ ]:
listings['price'].head() #few rows of price variable

,price
0,NaN
1,"$1,865.00"
2,$905.00
3,"$1,865.00"
4,"$1,800.00"


In [ ]:
listings['price'] = listings['price'].str.replace(r'[$,]', '', regex=True).astype(float) #removing dollar signs and commas and converting it to float

In [ ]:
# What is the distribution of 'price'?
print(listings['price'].describe())

count     13596.000000
mean       1372.736320
std        1403.526364
min         100.000000
25%         900.000000
50%        1186.000000
75%        1563.750000
max      100000.000000
Name: price, dtype: float64


In [ ]:
# 'price' column is in dkk currency and not us dollars or eur.
listings.rename(columns={'price': 'price_dkk'}, inplace=True)

In [ ]:
lower_threshold = 300 #setting minimum price threshold of 300 DKK (~ 40€) to exclude extremely low prices.
upper_threshold = 37303 #setting maximum price threshold of 37303 DKK (~ 5000€) to exclude extremely high prices.

# setting prices below 'lower_threshold' to 'lower_threshold' and above 'upper_threshold' to 'upper_threshold'.
listings['price_dkk'] = listings['price_dkk'].clip(lower= lower_threshold, upper= upper_threshold)
print(listings['price_dkk'].describe())

count    13596.000000
mean      1365.990953
std       1064.872166
min        300.000000
25%        900.000000
50%       1186.000000
75%       1563.750000
max      37303.000000
Name: price_dkk, dtype: float64


## Host Response Rate and Acceptance Rate

In [ ]:
listings[['host_response_rate', 'host_acceptance_rate']].head() #few rows

,host_response_rate,host_acceptance_rate
0,NaN,NaN
1,NaN,100%
2,100%,0%
3,100%,100%
4,90%,0%


In [ ]:
listings['host_response_rate'] = listings['host_response_rate'].str.replace(r'%', '', regex=True).astype(float)/100 #removing % sign and converting it to float
listings['host_acceptance_rate'] = listings['host_acceptance_rate'].str.replace(r'%', '', regex=True).astype(float)/100 #removing % sign and converting it to float
listings[['host_response_rate', 'host_acceptance_rate']].head() #few rows

,host_response_rate,host_acceptance_rate
0,NaN,NaN
1,NaN,1.0
2,1.0,0.0
3,1.0,1.0
4,0.9,0.0


## Data Type Adjustment

In [ ]:
listings['last_scraped'] = pd.to_datetime(listings['last_scraped']) #'last_scraped' column to datetime format
listings['host_since'] = pd.to_datetime(listings['host_since']) #'host_since' column to datetime format
listings['calendar_last_scraped'] = pd.to_datetime(listings['calendar_last_scraped']) #'calendar_last_scraped' column to datetime format
listings['first_review'] = pd.to_datetime(listings['first_review']) #'first_review' column to datetime format
listings['last_review'] = pd.to_datetime(listings['last_review']) #'last_review' column to datetime format

listings['host_is_superhost'] = listings['host_is_superhost'].map({'t': True, 'f': False}) #from 't' and 'f' to boolean (True/False)
listings['host_has_profile_pic'] = listings['host_has_profile_pic'].map({'t': True, 'f': False}) #from 't' and 'f' to boolean (True/False)
listings['host_identity_verified'] = listings['host_identity_verified'].map({'t': True, 'f': False}) #from 't' and 'f' to boolean (True/False)
listings['has_availability'] = listings['has_availability'].map({'t': True, 'f': False}) #from 't' and 'f' to boolean (True/False)
listings['instant_bookable'] = listings['instant_bookable'].map({'t': True, 'f': False}) #from 't' and 'f' to boolean (True/False)


## Handling Missing Values

In [ ]:
print(listings.isnull().sum()) #checking for missing values

id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
source                                             0
                                                ... 
calculated_host_listings_count                     0
calculated_host_listings_count_entire_homes        0
calculated_host_listings_count_private_rooms       0
calculated_host_listings_count_shared_rooms        0
reviews_per_month                               3220
Length: 75, dtype: int64


In [ ]:
listings.drop(['neighbourhood_group_cleansed', 'calendar_updated', 'license'], axis=1, inplace=True) #these variables only contains missing values

In [ ]:
listings['beds'] = listings['beds'].fillna(listings['bedrooms']) #set null values in 'beds' with the corresponding values from 'bedrooms' (hypothesis: if a listing has 2 bedrooms, it has for sure at least 2 beds)

## Exploring Individual Variables

In [ ]:
print(listings['neighbourhood_cleansed'].unique())

['Vesterbro-Kongens Enghave' 'Nrrebro' 'Amager st' 'sterbro' 'Indre By'
 'Bispebjerg' 'Amager Vest' 'Valby' 'Frederiksberg' 'Vanlse'
 'Brnshj-Husum']


In [ ]:
neighbourhood_corrections = {
    "Amager st": "Amager Øst",       # From "Amager st" to "Amager Øst"
    "Brnshj-Husum": "Brønshøj-Husum", # From "Brnshj–Husum" to "Brønshøj-Husum"
    "Nrrebro": "Nørrebro",            # From "Nrrebro" to "Nørrebro"
    "sterbro": "Østerbro",           # From "sterbro" to "Østerbro"
    "Vanlse": "Vanløse"              # From "Vanlse" to "Vanløse"
}
listings['neighbourhood_cleansed'] = listings['neighbourhood_cleansed'].replace(neighbourhood_corrections) #rename values in 'neighbourhood_cleansed' column

In [ ]:
print(listings['neighbourhood_cleansed'].unique())

['Vesterbro-Kongens Enghave' 'Nørrebro' 'Amager Øst' 'Østerbro' 'Indre By'
 'Bispebjerg' 'Amager Vest' 'Valby' 'Frederiksberg' 'Vanløse'
 'Brønshøj-Husum']


In [ ]:
print(listings['neighbourhood'].unique())

['Copenhagen, Capital Region of Denmark, Denmark' nan
 'Copenhagen, V, Denmark'
 'Frederiksberg, Capital Region of Denmark, Denmark' 'Copenhagen, Denmark'
 'Copenhagen, Hovedstaden, Denmark' 'Valby, Denmark'
 'Frederiksberg, Denmark' 'Valby, Capital Region of Denmark, Denmark'
 'København Ø, Capital Region of Denmark, Denmark'
 'Kobenhavn V, Capital Region of Denmark, Denmark'
 'Hellerup, Capital Region of Denmark, Denmark'
 'Bronshoj, Capital Region of Denmark, Denmark' 'København S, Denmark'
 'Vanløse, Capital Region of Denmark, Denmark'
 'Brønshøj, Capital Region of Denmark, Denmark' 'københavn V, Denmark'
 'kbh, Hovedstaden, Denmark' 'Copenhagen, Danmark, Denmark'
 'Copenhagen, S, Denmark'
 'Copenhagen , Capital Region of Denmark, Denmark'
 'Copenhagen, Region Zealand, Denmark' 'Copenhagen V, Denmark'
 'Copenhagen , Copenhagen, Denmark'
 'Copenhagen N.W, Capital Region of Denmark, Denmark'
 'Copenhagen N, Denmark' 'Søborg, Denmark' 'Copenhagen , Denmark'
 'Copenhagen NV, Denmark'
 

In [ ]:
# 'neighbourhood' contains information already represented (and improved) in neighborhood_cleansed
# so keeping it would be redundant
listings.drop('neighbourhood', axis=1, inplace=True)

In [ ]:
listings[['bathrooms', 'bathrooms_text']].head() #few rows

,bathrooms,bathrooms_text
0,NaN,1.5 baths
1,1.5,1.5 baths
2,0.0,0 baths
3,2.0,2 baths
4,1.0,1 bath


In [ ]:
def extract_numbers(text):
    if pd.notnull(text):  #if text is not null
        if re.search(r'\d+(\.\d+)?', text): #if find int or float numbers
            return float(re.search(r'\d+(\.\d+)?', text).group(0))  #return number as a float
    return None  #return None if text is null or does not contain numbers

listings['bathrooms'] = listings['bathrooms'].fillna(listings['bathrooms_text'].apply(extract_numbers)) #fill null values of bathrooms with numbers in bathrooms_text

In [ ]:
listings[['bathrooms', 'bathrooms_text']].head() #few rows

,bathrooms,bathrooms_text
0,1.5,1.5 baths
1,1.5,1.5 baths
2,0.0,0 baths
3,2.0,2 baths
4,1.0,1 bath


In [ ]:
print(listings['bathrooms_text'].unique())

['1.5 baths' '0 baths' '2 baths' '1 bath' '1 shared bath' '3 baths'
 '1 private bath' '2.5 baths' 'Half-bath' nan 'Shared half-bath'
 'Private half-bath' '1.5 shared baths' '3.5 baths' '2 shared baths'
 '5 baths' '0 shared baths' '4 baths' '2.5 shared baths' '3 shared baths'
 '8 baths']


In [ ]:
listings['is_bathroom_shared'] = listings['bathrooms_text'].str.contains('shared', case=False) #if "shared" in "bathroom_text" then "is_bathroom_shared" = True, else False


In [ ]:
#get a numeric summary of all the numeric variables, is it everything ok?
numeric_summary = listings.describe()

print(numeric_summary.iloc[:, 0:15])
print(numeric_summary.iloc[:, 15:30])
print(numeric_summary.iloc[:, 30:46])

                 id     scrape_id                   last_scraped  \
count  2.090900e+04  2.090900e+04                          20909   
mean   5.137813e+17  2.024063e+13  2024-06-29 16:49:50.329523200   
min    3.109400e+04  2.024063e+13            2024-06-29 00:00:00   
25%    3.140450e+07  2.024063e+13            2024-06-29 00:00:00   
50%    6.452861e+17  2.024063e+13            2024-06-30 00:00:00   
75%    9.441256e+17  2.024063e+13            2024-06-30 00:00:00   
max    1.189116e+18  2.024063e+13            2024-06-30 00:00:00   
std    4.737075e+17  1.914108e+00                            NaN   

            host_id                     host_since  host_response_rate  \
count  2.090900e+04                          20908        14441.000000   
mean   1.442690e+08  2016-11-19 18:39:56.785919488            0.898103   
min    5.130000e+02            2008-06-27 00:00:00            0.000000   
25%    1.739651e+07            2014-07-06 00:00:00            1.000000   
50%    6.431141e+

In [ ]:
# Corrections to strings

listings['description'] = listings['description'].str.replace(r'<br\s*/?>', ' ', regex=True)
listings['description'] = listings['description'].str.replace(r'\s+', ' ', regex=True)

listings['neighborhood_overview'] = listings['neighborhood_overview'].str.replace(r'<br\s*/?>', ' ', regex=True)
listings['neighborhood_overview'] = listings['neighborhood_overview'].str.replace(r'\s+', ' ', regex=True)

listings['host_about'] = listings['host_about'].str.replace(r'<br\s*/?>', ' ', regex=True)
listings['host_about'] = listings['host_about'].str.replace(r'\s+', ' ', regex=True)

In [ ]:
listings.to_csv('/content/drive/MyDrive/Airbnb/Airbnb_cleaned_csvs/listings_cleaned_with_missing_prices.csv', index=False)